# Setup

In [1]:
import pandas as pd
import numpy as np
import os, time, stat, io, glob, pickle
from scipy.stats import gamma, norm
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('fivethirtyeight')
import plotly.io as pio

%load_ext autoreload
%autoreload 2
from covid_data_helper import *
from coronita_chart_helper import *
from coronita_model_helper import *
from coronita_web_helper import *

from matplotlib.backends.backend_pdf import PdfPages


In [2]:
## DATA INGESTION ##

df_st_testing = get_covid19_tracking_data()

df_census = get_census_pop()

df_counties = get_complete_county_data()

counties_geo = get_counties_geo()

df_jhu_counties = get_jhu_counties()

df_st_testing_fmt = df_st_testing.copy()
df_st_testing_fmt = df_st_testing_fmt.rename(columns={'death':'deaths','positive':'cases'}).unstack('code')

df_interventions = get_state_policy_events()

df_goog_mob_us = get_goog_mvmt_us()
df_goog_mob_us = df_goog_mob_us[df_goog_mob_us.state.isnull()].set_index('dt')

df_goog_mob_state = get_goog_mvmt_state()

## MODEL PARAMETERS ##

covid_params = {}
covid_params['d_incub'] = 3.
covid_params['d_infect'] = 4.
covid_params['mort_rt'] = 0.01
covid_params['d_in_hosp'] = 11
covid_params['hosp_rt'] = 0.04
covid_params['d_to_hosp'] = 7.0
covid_params['d_in_hosp_mild'] = 11.0
covid_params['icu_rt'] = 13./41.
covid_params['d_in_icu'] = 13.0
covid_params['vent_rt'] = 0.4
covid_params['d_til_death'] =  17.0
covid_params['policy_trigger'] = True
covid_params['policy_trigger_once'] = True


State Testing Data Last Observation:  20200714


In [3]:
list_of_files = glob.glob('./output/df_fore_allstates_*.pkl') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)
df_fore_allstates = pd.read_pickle(latest_file)


./output/df_fore_allstates_20200715.pkl


In [4]:
list_of_files = glob.glob('./output/allstate_model_dicts_*.pkl') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)
with open(latest_file, 'rb') as handle:
    allstate_model_dicts = pickle.load(handle)

./output/allstate_model_dicts_20200715.pkl


# Development

In [24]:
state = 'NY'
model_dict = allstate_model_dicts[state]
df_agg = df_fore_allstates[state].unstack('metric')
scenario_name = 'No Change in Future Rᵗ Until 20% Hospital Capacity Trigger'
chart_title = "{1} Scenario".format(
        model_dict['region_name'], scenario_name)
param_str = param_str_maker(model_dict)


In [63]:
reset_output()
output_notebook()

s1 = bk_bar_and_line_chart(bar_series=model_dict['df_hist']['cases_daily'].dropna(how='all'),
                       bar_name='# of Positive Tests', bar_color='#e5ae38',
                       line_series=model_dict['df_hist']['cases_daily'].rolling(7, min_periods=1).mean(),
                       line_name='7-Day Rolling Average', yformat='{:0,.0f}',
                       chart_title='{}: Positive COVID-19 Tests Per Day'.format(model_dict['region_name'])
                       )

s2 = bk_bar_and_line_chart(bar_series=model_dict['df_hist']['pos_neg_tests_daily'].dropna(how='all'),
                       bar_name='# of Negative Tests',
                       line_series=model_dict['df_hist']['pos_neg_tests_daily'].rolling(7, min_periods=1).mean(),
                       line_name='All Tests (7-Day Rolling Average)', yformat='{:0,.0f}',
                       chart_title='{}: COVID-19 Tests Per Day'.format(model_dict['region_name']),
                       bar2_series=model_dict['df_hist']['cases_daily'], bar2_name='# of Positive Tests'
                       )
s2.x_range = s1.x_range

s3 = bk_postestshare(model_dict)
s3.x_range = s1.x_range

output_notebook(); curdoc().theme = bk_theme
layout = column(row(s1, s2), row(s3), sizing_mode='scale_width')
show(layout)
output_file('./output/bokeh.html'); curdoc().theme = bk_theme
save(layout)


Loading BokehJS ...

Loading BokehJS ...

'/Users/mdonnelly/repos/coronita/output/bokeh.html'

weighted_average                                        
metric                   rt    rt_l68    rt_l95    rt_u68    rt_u95
dt                                                                 
2020-03-04         3.155678  2.679500  2.222369  3.631857  4.088988
2020-03-05         3.108127  2.636401  2.183544  3.579854  4.032711
2020-03-06         3.026840  2.591759  2.174081  3.461922  3.879600
2020-03-07         2.953799  2.580395  2.221928  3.327202  3.685670
2020-03-08         2.852916  2.230027  1.632054  3.475805  4.073779
...                     ...       ...       ...       ...       ...
2020-07-03         0.989004  0.770562  0.560857  1.207447  1.417152
2020-07-04         1.006521  0.793351  0.588707  1.219692  1.424336
2020-07-05         1.032626  0.809408  0.595118  1.255845  1.470134
2020-07-06         1.068976  0.830129  0.600836  1.307823  1.537116
2020-07-07         1.089973  0.853869  0.627210  1.326077  1.552736

[126 rows x 5 columns]

In [70]:
model_dict['df_rts_conf'][['weighted_average']].unstack('metric')

weighted_average                                        
metric                   rt    rt_l68    rt_l95    rt_u68    rt_u95
dt                                                                 
2020-03-04         3.155678  2.679500  2.222369  3.631857  4.088988
2020-03-05         3.108127  2.636401  2.183544  3.579854  4.032711
2020-03-06         3.026840  2.591759  2.174081  3.461922  3.879600
2020-03-07         2.953799  2.580395  2.221928  3.327202  3.685670
2020-03-08         2.852916  2.230027  1.632054  3.475805  4.073779
...                     ...       ...       ...       ...       ...
2020-07-03         0.989004  0.770562  0.560857  1.207447  1.417152
2020-07-04         1.006521  0.793351  0.588707  1.219692  1.424336
2020-07-05         1.032626  0.809408  0.595118  1.255845  1.470134
2020-07-06         1.068976  0.830129  0.600836  1.307823  1.537116
2020-07-07         1.089973  0.853869  0.627210  1.326077  1.552736

[126 rows x 5 columns]

In [85]:
str(10)+'pt'

'10pt'

In [128]:
from coronita_chart_helper import *
reset_output()
output_notebook()

p = bk_rt_confid(model_dict, param_str, chart_title)

output_notebook(); curdoc().theme = bk_theme

show(p)

Loading BokehJS ...

Loading BokehJS ...

In [88]:
from coronita_chart_helper import *
reset_output()
output_notebook()

p = bk_population_share(df_agg, model_dict, param_str, chart_title)

# bk_add_event_lines(p, 
#                    df_interventions[df_interventions.state_code.isin([state, 'US'])].groupby('dt').first().reset_index())

output_notebook(); curdoc().theme = bk_theme

show(p)

Loading BokehJS ...

Loading BokehJS ...

In [107]:
from coronita_chart_helper import *
reset_output()
output_notebook()

df_chart = df_fore_allstates.stack().unstack('metric')['exposed_daily'].unstack(1)
names_sub = df_chart.columns.to_list()
data = df_chart.reset_index()
source = ColumnDataSource(data)
# create CDS for filtered sources
filt_data1 = data[['dt','NY']].rename(columns={'NY':'cases'})
src2 = ColumnDataSource(filt_data1)
filt_data2 = data[['dt','CA']].rename(columns={'CA':'cases'})
src3 = ColumnDataSource(filt_data2)

hover_tool = HoverTool(tooltips=[
            ('Cases', '@cases'),
            ('Date', '@dt')],
            formatters={'dt': 'datetime'}
        )

p1 = figure(x_axis_type='datetime',
           tools=[hover_tool],title='Sars-CoV-2 cases (ECDC)',
            y_range=Range1d(start=0, end=filt_data1.cases.max()+50),
            sizing_mode="scale_width", plot_height=400
           )
p1.line(x='dt',y='cases', source=src2, legend_label="State 1 (Left Axis)", line_color='blue',
        line_width=3,line_alpha=.8)
#set the second y-axis and use that with our second line
p1.extra_y_ranges = {"y2": Range1d(start=0, end=filt_data2.cases.max()+50)}
p1.add_layout(LinearAxis(y_range_name="y2",
                        formatter=NumeralTickFormatter(format="0%")
                        ),
              'right')
p1.line(x='dt',y='cases', source=src3, legend_label="State 2 (Right Axis)", line_color='orange',
        line_width=3,line_alpha=.8,y_range_name="y2")

p1.yaxis[0].axis_label = 'NY'
p1.yaxis[1].axis_label = 'CA'
p1.yaxis.formatter = NumeralTickFormatter(format="0a")
p1.legend.location = "top_left"
p1.xaxis.axis_label = 'Date'
p1.xaxis.formatter=DatetimeTickFormatter(days='%b %d',
months='%b %d',
)
#this javascript snippet is the callback when either select is changed

# y_range.end = parseInt(y[y.length - 1]+50);
code="""
var c = cb_obj.value;
ax.axis_label = c;
var y = s1.data[c];
const y_nonan = y.filter(function (value) {
    return !Number.isNaN(value);
});
var y_max = Math.max(...y_nonan);
s2.data['cases'] = y;
y_range.start = 0;
y_range.end = parseInt(y_max);
s2.change.emit();
"""
callback1 = CustomJS(args=dict(s1=source,
                               s2=src2,
                               y_range=p1.y_range,
                               ax=p1.yaxis[0],
                               leg=p1.legend
                              ), code=code)
callback2 = CustomJS(args=dict(s1=source,
                               s2=src3,
                               y_range=p1.extra_y_ranges['y2'],
                               ax=p1.yaxis[1],
                               leg=p1.legend
                              ), code=code)
# names = list(df.countriesAndTerritories.unique() )
# names_sub=['NY','United_Kingdom','United_States_of_America','Spain','Italy',
#            'Germany','France','Iran','Australia','CA','Sweden','Belgium','Turkey','India']
select1 = Select(title="State 1:", value='NY', options=names_sub)
select1.js_on_change('value', callback1)
select2 = Select(title="State 2:", value='CA', options=names_sub)
select2.js_on_change('value', callback2)
btn = Button(label='Update')

output_notebook(); curdoc().theme = bk_theme
layout = column(row(select1,select2), row(p1), sizing_mode='scale_width')
show(layout)
output_file('./output/bokeh.html'); curdoc().theme = bk_theme
save(layout)

Loading BokehJS ...

Loading BokehJS ...

'/Users/mdonnelly/repos/coronita/output/bokeh.html'